In [0]:
import os
import re
import collections
import numpy as np

In [0]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, CuDNNLSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [0]:
def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()
    return data.split('\n')

def process_data(data):
    new_sentence = []
    for sentence in data:
        clean = re.compile('<.*>')
        new_sentence.append(re.sub(clean, '', sentence))
    del new_sentence[-2:]
    return new_sentence


In [0]:
english = process_data(load_data('/content/small_vocab_en.txt'))
french = process_data(load_data('/content/small_vocab_fr.txt'))

In [5]:
english_words_counter = collections.Counter([word for sentence in english for word in sentence.split(" ")])
print("Total no. of words in english: ", len([word for sentence in english for word in sentence.split(" ")]))
print("Total no. of unique words: ", len(english_words_counter))

french_words_counter = collections.Counter([word for sentence in french for word in sentence.split(" ")])
print("Total no. of words in french: ", len([word for sentence in french for word in sentence.split(" ")]))
print("Total no. of unique words: ", len(french_words_counter))

Total no. of words in english:  1823292
Total no. of unique words:  228
Total no. of words in french:  1961298
Total no. of unique words:  356


In [0]:
def tokenize(x):
    tk = Tokenizer(char_level = False)
    tk.fit_on_texts(x)
    return tk.texts_to_sequences(x), tk

def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

In [0]:
def preprocess(x, y):
    data_x, x_tk = tokenize(x)
    data_y, y_tk = tokenize(y)

    data_x = pad(data_x)
    data_y = pad(data_y)

    data_y = data_y.reshape(*data_y.shape, 1)

    return data_x, data_y, x_tk, y_tk

In [0]:
processed_english, processed_french, english_tokenizer, french_tokenizer = preprocess(english, french)

In [9]:
max_english_sequence_length = processed_english.shape[1]
max_french_sequence_length = processed_french.shape[1]
print(max_english_sequence_length)
print(max_french_sequence_length)
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

15
21


In [0]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

#**Simple RNN**



In [0]:
def simple_model(input_shape, french_vocab_size):
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, optimizer = Adam(1e-3), metrics = ['accuracy'])
    return model

In [0]:
tmp_x = pad(processed_english, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, processed_french.shape[-2], 1))

In [13]:
simple_rnn_model = simple_model(tmp_x.shape, french_vocab_size)

W0625 08:47:12.225424 140256952489856 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
simple_rnn_model.fit(tmp_x, processed_french, batch_size=1024, epochs=30, validation_split=0.2)

W0625 08:47:12.639853 140256952489856 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 110288 samples, validate on 27572 samples
Epoch 1/30
110288/110288 [==============================] - 8s 77us/sample - loss: 3.5501 - acc: 0.4149 - val_loss: nan - val_acc: 0.4718
Epoch 2/30
110288/110288 [==============================] - 8s 70us/sample - loss: 2.3920 - acc: 0.4763 - val_loss: nan - val_acc: 0.5028
Epoch 3/30
110288/110288 [==============================] - 8s 70us/sample - loss: 2.1528 - acc: 0.5314 - val_loss: nan - val_acc: 0.5567
Epoch 4/30
110288/110288 [==============================] - 8s 70us/sample - loss: 1.9074 - acc: 0.5661 - val_loss: nan - val_acc: 0.5820
Epoch 5/30
110288/110288 [==============================] - 8s 71us/sample - loss: 1.7454 - acc: 0.5827 - val_loss: nan - val_acc: 0.5855
Epoch 6/30
110288/110288 [==============================] - 8s 71us/sample - loss: 1.6439 - acc: 0.5910 - val_loss: nan - val_acc: 0.5953
Epoch 7/30
110288/110288 [==============================] - 8s 70us/sample - loss: 1.5732 - acc: 0.5996 - val_loss: nan -

In [15]:
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme en mois de et il il est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


#**Using Embeddings**

In [0]:
def embedding_model(input_shape, french_vocab_size):
    embedding = Embedding(french_vocab_size, 64, input_length = input_shape[1])
    rnn = GRU(64, return_sequences = True, activation='tanh')
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    
    model = Sequential()
    
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(1e-3),
                  metrics=['accuracy'])
    return model

In [0]:
tmp_x = pad(processed_english, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, processed_french.shape[-2]))

In [18]:
simple_embedd_model = embedding_model(tmp_x.shape, french_vocab_size)

W0625 08:51:08.874375 140256952489856 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [19]:
simple_embedd_model.fit(tmp_x, processed_french, batch_size=1024, epochs=10, validation_split=0.2)

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
110288/110288 [==============================] - 9s 77us/sample - loss: 3.7455 - acc: 0.4017 - val_loss: nan - val_acc: 0.4093
Epoch 2/10
110288/110288 [==============================] - 8s 73us/sample - loss: 2.6926 - acc: 0.4386 - val_loss: nan - val_acc: 0.5078
Epoch 3/10
110288/110288 [==============================] - 8s 74us/sample - loss: 2.0390 - acc: 0.5560 - val_loss: nan - val_acc: 0.6167
Epoch 4/10
110288/110288 [==============================] - 8s 73us/sample - loss: 1.4589 - acc: 0.6522 - val_loss: nan - val_acc: 0.6931
Epoch 5/10
110288/110288 [==============================] - 8s 73us/sample - loss: 1.1477 - acc: 0.7224 - val_loss: nan - val_acc: 0.7492
Epoch 6/10
110288/110288 [==============================] - 8s 73us/sample - loss: 0.9501 - acc: 0.7670 - val_loss: nan - val_acc: 0.7823
Epoch 7/10
110288/110288 [==============================] - 8s 73us/sample - loss: 0.8123 - acc: 0.7918 - val_loss: nan -

In [20]:
print(logits_to_text(simple_embedd_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme en l' et il il est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


#**Bidirectional RNN**

In [0]:
def simple_bidirectional_model(input_shape, french_vocab_size):
    
    model = Sequential()
    
    bdd = Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), input_shape = input_shape[1:])
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    
    model.add(bdd)
    model.add(logits)
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(1e-3), 
                 metrics = ['accuracy'])
    
    return model

In [0]:
tmp_x = pad(processed_english, processed_french.shape[1])
tmp_x = tmp_x.reshape((-1, processed_french.shape[-2], 1))

In [23]:
simple_bdd_model = simple_bidirectional_model(tmp_x.shape,  french_vocab_size+1)

W0625 08:52:32.487078 140256952489856 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0625 08:52:32.489733 140256952489856 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0625 08:52:32.491825 140256952489856 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is de

In [24]:
simple_bdd_model.fit(tmp_x, processed_french, batch_size=1024, epochs=20, validation_split=0.2)

Train on 110288 samples, validate on 27572 samples
Epoch 1/20
110288/110288 [==============================] - 15s 133us/sample - loss: 2.7254 - acc: 0.4996 - val_loss: 1.7869 - val_acc: 0.5696
Epoch 2/20
110288/110288 [==============================] - 14s 123us/sample - loss: 1.6539 - acc: 0.5870 - val_loss: 1.4756 - val_acc: 0.6108
Epoch 3/20
110288/110288 [==============================] - 14s 123us/sample - loss: 1.4474 - acc: 0.6146 - val_loss: 1.3557 - val_acc: 0.6266
Epoch 4/20
110288/110288 [==============================] - 14s 123us/sample - loss: 1.3377 - acc: 0.6318 - val_loss: 1.2920 - val_acc: 0.6365
Epoch 5/20
110288/110288 [==============================] - 14s 123us/sample - loss: 1.2631 - acc: 0.6454 - val_loss: 1.2606 - val_acc: 0.6337
Epoch 6/20
110288/110288 [==============================] - 14s 123us/sample - loss: 1.2077 - acc: 0.6563 - val_loss: 1.2437 - val_acc: 0.6325
Epoch 7/20
110288/110288 [==============================] - 14s 123us/sample - loss: 1.1634

In [25]:
print(logits_to_text(simple_bdd_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois froid au janvier mais il est agréable en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


#***Encoder Decoder***

In [0]:
def encdec_model(input_shape, output_sequence_length, french_vocab_size):
    
    model = Sequential()
    
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))

    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(1e-3), 
                 metrics = ['accuracy'])
    return model

In [0]:
tmp_x = pad(processed_english)
tmp_x = tmp_x.reshape((-1, processed_english.shape[1], 1))

In [0]:
enco_deco_model = encdec_model(
    tmp_x.shape,
    processed_french.shape[1],
    french_vocab_size+1)

In [30]:
enco_deco_model.fit(tmp_x, processed_french, batch_size=1024, epochs=20, validation_split=0.2)

Train on 110288 samples, validate on 27572 samples
Epoch 1/20
110288/110288 [==============================] - 14s 125us/sample - loss: 3.0118 - acc: 0.4402 - val_loss: 2.4269 - val_acc: 0.4955
Epoch 2/20
110288/110288 [==============================] - 13s 116us/sample - loss: 2.2445 - acc: 0.5017 - val_loss: 2.0706 - val_acc: 0.5111
Epoch 3/20
110288/110288 [==============================] - 13s 115us/sample - loss: 1.9764 - acc: 0.5168 - val_loss: 1.9026 - val_acc: 0.5255
Epoch 4/20
110288/110288 [==============================] - 13s 115us/sample - loss: 1.8427 - acc: 0.5383 - val_loss: 1.7807 - val_acc: 0.5491
Epoch 5/20
110288/110288 [==============================] - 13s 116us/sample - loss: 1.7260 - acc: 0.5587 - val_loss: 1.6800 - val_acc: 0.5644
Epoch 6/20
110288/110288 [==============================] - 13s 116us/sample - loss: 1.6402 - acc: 0.5721 - val_loss: 1.6030 - val_acc: 0.5772
Epoch 7/20
110288/110288 [==============================] - 13s 115us/sample - loss: 1.5785

In [31]:
print(logits_to_text(enco_deco_model.predict(tmp_x[:10])[2], french_tokenizer))

la est jamais agréable en mois et il est est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


# *Custom*

Create a model that incorporates embedding and a bidirectional RNN into one model.

In [0]:
def final_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
    model = Sequential()
    
    embed_layer = Embedding(input_dim = english_vocab_size, output_dim = 128, input_length = input_shape[1])
    bd_layer_1 = Bidirectional(GRU(256, return_sequences = False))
    repeat_vector_1 = RepeatVector(output_sequence_length)
    bd_layer_2 = Bidirectional(GRU(256, return_sequences = True))
    time_distributed = TimeDistributed(Dense(french_vocab_size, activation = 'softmax'))
    
    model.add(embed_layer)
    model.add(bd_layer_1)
    model.add(repeat_vector_1)
    model.add(bd_layer_2)
    model.add(time_distributed)
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(5e-3), 
                 metrics = ['accuracy'])
    
    return model

In [0]:
tmp_x = pad(processed_english)

In [0]:
custom_model = final_model(tmp_x.shape, processed_french.shape[1], english_vocab_size + 1, french_vocab_size + 1)

In [35]:
custom_model.fit(tmp_x, processed_french, batch_size = 1024, epochs = 18, validation_split = 0.2)

Train on 110288 samples, validate on 27572 samples
Epoch 1/18
110288/110288 [==============================] - 42s 380us/sample - loss: 2.0001 - acc: 0.5325 - val_loss: 1.3340 - val_acc: 0.6487
Epoch 2/18
110288/110288 [==============================] - 40s 361us/sample - loss: 1.0771 - acc: 0.6994 - val_loss: 0.8941 - val_acc: 0.7406
Epoch 3/18
110288/110288 [==============================] - 40s 360us/sample - loss: 0.7234 - acc: 0.7854 - val_loss: 0.5657 - val_acc: 0.8275
Epoch 4/18
110288/110288 [==============================] - 40s 360us/sample - loss: 0.4645 - acc: 0.8608 - val_loss: 0.3389 - val_acc: 0.9027
Epoch 5/18
110288/110288 [==============================] - 40s 364us/sample - loss: 0.2702 - acc: 0.9221 - val_loss: 0.2201 - val_acc: 0.9360
Epoch 6/18
110288/110288 [==============================] - 40s 359us/sample - loss: 0.1796 - acc: 0.9480 - val_loss: 0.1561 - val_acc: 0.9555
Epoch 7/18
110288/110288 [==============================] - 40s 361us/sample - loss: 0.1394

In [36]:
print(logits_to_text(custom_model.predict(tmp_x[:10])[2], french_tokenizer))

california est généralement calme en mars et il est généralement chaud en juin <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


# *LSTM*

In [0]:
def lstm_model(input_shape, french_vocab_size):
    
    model = Sequential()
    
    embedding = Embedding(french_vocab_size, 64, input_length = input_shape[1])
    lstm_layer_1 = CuDNNLSTM(64, return_sequences = True)
    logits = TimeDistributed(Dense(french_vocab_size, activation = 'softmax'))
    
    model.add(embedding)
    model.add(lstm_layer_1)
    model.add(logits)
    
    model.compile(loss = sparse_categorical_crossentropy, optimizer = Adam(5e-3), metrics = ['accuracy'])
    return model

In [0]:
tmp_x = pad(processed_english, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, processed_french.shape[-2]))

In [0]:
simple_lstm_model = lstm_model(tmp_x.shape, french_vocab_size)

In [46]:
simple_lstm_model.fit(tmp_x, processed_french, batch_size=1024, epochs=80, validation_split=0.2)

Train on 110288 samples, validate on 27572 samples
Epoch 1/80
110288/110288 [==============================] - 4s 39us/sample - loss: 2.8838 - acc: 0.4525 - val_loss: nan - val_acc: 0.5536
Epoch 2/80
110288/110288 [==============================] - 3s 30us/sample - loss: 1.4746 - acc: 0.6364 - val_loss: nan - val_acc: 0.7236
Epoch 3/80
110288/110288 [==============================] - 3s 30us/sample - loss: 0.8894 - acc: 0.7675 - val_loss: nan - val_acc: 0.7969
Epoch 4/80
110288/110288 [==============================] - 3s 30us/sample - loss: 0.6417 - acc: 0.8168 - val_loss: nan - val_acc: 0.8364
Epoch 5/80
110288/110288 [==============================] - 3s 30us/sample - loss: 0.5209 - acc: 0.8462 - val_loss: nan - val_acc: 0.8584
Epoch 6/80
110288/110288 [==============================] - 3s 30us/sample - loss: 0.4487 - acc: 0.8652 - val_loss: nan - val_acc: 0.8723
Epoch 7/80
110288/110288 [==============================] - 3s 30us/sample - loss: 0.4016 - acc: 0.8778 - val_loss: nan -

In [47]:
print(logits_to_text(simple_lstm_model.predict(tmp_x[:1])[0], french_tokenizer))

new jersey est parfois calme en l' et et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
